In [46]:
import pandas as pd
import numpy as np

In [47]:
file='model_template.xlsm'

In [48]:
ingredientes_df = pd.read_excel(io=file, sheet_name='ingredientes')
ingredientes_df.head()
ingredientes_list = list(ingredientes_df['nombre'].unique())
print(ingredientes_list)


['maiz', 'tsoya', 'destilado', 'gluten', 'destiladohp', 'forraje', 'cascarilla', 'frijol', 'tgirasol', 'trigo']


In [49]:
plantas_df = pd.read_excel(io=file, sheet_name='plantas')
plantas_list = list(plantas_df['planta'].unique())
print(plantas_list)

['envigado', 'cartago', 'neiva', 'bogota', 'launion', 'barbosa', 'ibague', 'mosquera', 'buga', 'itagui', 'bmanga', 'cienaga', 'pimpollo']


In [51]:
# Obtener consumo
consumo_df = pd.read_excel(io=file, sheet_name='consumo_proyectado')
indexvar = ['planta', 'ingrediente']
consumo_columns = list(consumo_df.drop(columns=indexvar).columns)
consumo_df['promedio'] = consumo_df[consumo_columns].apply(np.mean, axis=1)
consumo_df = consumo_df[indexvar + ['promedio']]
consumo_df.head()

,planta,ingrediente,promedio
0,envigado,maiz,365557.478125
1,envigado,tsoya,97489.765000
2,envigado,destilado,48530.107500
3,envigado,gluten,4309.328750
4,envigado,destiladohp,5732.804375


In [53]:
# Obtener minumo
safety_stock_df = pd.read_excel(io=file,sheet_name='safety_stock')

In [68]:
# Obtener inventario actual
inventario_df = pd.read_excel(io=file, sheet_name='unidades_almacenamiento')
inventario_df = inventario_df[inventario_df['ingrediente_actual'].isin(ingredientes_list)].copy()
inventario_df = inventario_df.groupby(['planta', 'ingrediente_actual'])[['cantidad_actual']].sum().reset_index()
inventario_df.rename(columns={'ingrediente_actual':'ingrediente', 'cantidad_actual':'inventario'}, inplace=True)
inventario_df.head()


,planta,ingrediente,inventario
0,barbosa,maiz,261661.2
1,barbosa,tsoya,132227.2
2,bmanga,cascarilla,0.0
3,bmanga,destilado,56134.5
4,bmanga,destiladohp,19860.0


In [75]:
# Calcular capacidad actual
def capacidad_actual(unidades_asignadas_df):

    capacidad_actual_df = unidades_asignadas_df[unidades_asignadas_df['ingrediente_actual'].isin(ingredientes_list)].copy()

    capacidad_actual_df['capacidad'] = capacidad_actual_df.apply(lambda x: x[x['ingrediente_actual']] , axis=1)

    capacidad_actual_df = capacidad_actual_df.groupby(['planta', 'ingrediente_actual'])[['capacidad']].sum().reset_index()

    capacidad_actual_df.rename(columns={'ingrediente_actual':'ingrediente'}, inplace=True)

    return capacidad_actual_df

In [107]:
def calcular(consumo, safety_stock, inventario, capacidad_actual):
    # Generar la tabla de resultados
    solucion_dict = dict()
    solucion_dict['planta'] = list()
    solucion_dict['ingrediente'] = list()

    for planta in plantas_list:
        for ingrediente in ingredientes_list:
            solucion_dict['planta'].append(planta)
            solucion_dict['ingrediente'].append(ingrediente)

    solucion_df = pd.DataFrame(solucion_dict)

    # Agregar consumo a solucion_df
    solucion_df = pd.merge(left=solucion_df,
                        right=consumo,
                        left_on=['planta', 'ingrediente'],
                        right_on=['planta', 'ingrediente'],
                        how='left')
    # Agregar las columnas de dias ss y capaacidad de recepcion
    solucion_df = pd.merge(left=solucion_df,
                        right=safety_stock,
                        left_on=['planta', 'ingrediente'],
                        right_on=['planta', 'ingrediente'],
                        how='left')

    # Agregar las columnas de inventario actual
    solucion_df = pd.merge(left=solucion_df,
                        right=inventario,
                        left_on=['planta', 'ingrediente'],
                        right_on=['planta', 'ingrediente'],
                        how='left')

    # Agregar las columnas de capacidad actual
    solucion_df = pd.merge(left=solucion_df,
                        right=capacidad_actual,
                        left_on=['planta', 'ingrediente'],
                        right_on=['planta', 'ingrediente'],
                        how='left').fillna(0.0)
    solucion_df.head()
    solucion_df['minimo'] = solucion_df['dias_ss']*solucion_df['promedio']
    solucion_df['capacidad_dias'] = solucion_df.apply(lambda x: x['capacidad']/x['promedio'] if x['promedio']>0 else 1000, axis=1)

    solucion_df.sort_values(by=['capacidad_dias', 'promedio'], ascending=[True, False], inplace=True)

    solucion_df.reset_index(inplace=True)


    return solucion_df

In [140]:
# Leer unidades
unidades_df = pd.read_excel(io=file, sheet_name='unidades_almacenamiento')
unidades_df['ingrediente_actual'] = unidades_df.apply(lambda x:x['ingrediente_actual'] if x['cantidad_actual']>0 else '', axis=1)

In [141]:
capacidad_actual_df = capacidad_actual(unidades_df)
df = calcular(consumo=consumo_df, safety_stock=safety_stock_df, inventario=inventario_df, capacidad_actual=capacidad_actual_df)
df.head(10)

,index,planta,ingrediente,promedio,dias_ss,Capacidad_recepcion_kg,inventario,capacidad,minimo,capacidad_dias
0,15,cartago,forraje,12651.352500,0.0,510000.0,0.0,0.0,0.000000,0.0
1,116,cienaga,cascarilla,7664.114375,5.0,408000.0,0.0,0.0,38320.571875,0.0
2,73,mosquera,gluten,6527.781875,0.0,442000.0,0.0,0.0,0.000000,0.0
3,14,cartago,destiladohp,6388.933125,0.0,510000.0,0.0,0.0,0.000000,0.0
4,85,buga,forraje,6237.256875,0.0,748000.0,0.0,0.0,0.000000,0.0
5,94,itagui,destiladohp,4488.206875,0.0,272000.0,0.0,0.0,0.000000,0.0
6,3,envigado,gluten,4309.328750,0.0,510000.0,0.0,0.0,0.000000,0.0
7,34,bogota,destiladohp,4199.458750,0.0,306000.0,0.0,0.0,0.000000,0.0
8,74,mosquera,destiladohp,3734.112500,0.0,306000.0,0.0,0.0,0.000000,0.0
9,103,bmanga,gluten,3572.561250,0.0,425000.0,0.0,0.0,0.000000,0.0


In [142]:
values = list(unidades_df['ingrediente_actual'])
for i in unidades_df.index:
    if i in unidades_df[unidades_df['cantidad_actual']<=0].index:
        # Obtener información sobre lo que puede guardar la primera unidad vacia
        planta = unidades_df.loc[i]['planta']
        unidad = unidades_df.loc[i]['unidad_almacenamiento']
        posibles = [x for x in ingredientes_list if unidades_df.loc[i][x] > 0]

        # Totalizar la capacidad actual
        capacidad_actual_df = capacidad_actual(unidades_df)

        # Calcular capacidad en dias
        df = calcular(consumo=consumo_df, safety_stock=safety_stock_df, inventario=inventario_df, capacidad_actual=capacidad_actual_df)

        # Obtener el ingrediente con menor cantidad de capacidad en días
        ingrediente_a_asignar = df[(df['planta']==planta)&(df['ingrediente'].isin(posibles))].iloc[0]['ingrediente']
        #print('trabajando con', planta, unidad, posibles, ingrediente_a_asignar)

        # Asignar la unidad vacia al ingrediente
        values[i] = ingrediente_a_asignar
        unidades_df['ingrediente_actual'] = values


In [143]:
capacidad_actual_df = capacidad_actual(unidades_df)
df = calcular(consumo=consumo_df, safety_stock=safety_stock_df, inventario=inventario_df, capacidad_actual=capacidad_actual_df)
df.head(10)

,index,planta,ingrediente,promedio,dias_ss,Capacidad_recepcion_kg,inventario,capacidad,minimo,capacidad_dias
0,14,cartago,destiladohp,6388.933125,0.0,510000.0,0.0,0.0,0.0,0.0
1,85,buga,forraje,6237.256875,0.0,748000.0,0.0,0.0,0.0,0.0
2,3,envigado,gluten,4309.328750,0.0,510000.0,0.0,0.0,0.0,0.0
3,34,bogota,destiladohp,4199.458750,0.0,306000.0,0.0,0.0,0.0,0.0
4,103,bmanga,gluten,3572.561250,0.0,425000.0,0.0,0.0,0.0,0.0
5,25,neiva,forraje,2893.316250,0.0,238000.0,0.0,0.0,0.0,0.0
6,24,neiva,destiladohp,2319.111250,0.0,340000.0,0.0,0.0,0.0,0.0
7,114,cienaga,destiladohp,810.126250,0.0,408000.0,0.0,0.0,0.0,0.0
8,35,bogota,forraje,669.353750,0.0,238000.0,0.0,0.0,0.0,0.0
9,53,barbosa,gluten,474.719375,0.0,238000.0,0.0,0.0,0.0,0.0


In [136]:
df

,index,planta,ingrediente,promedio,dias_ss,Capacidad_recepcion_kg,inventario,capacidad,minimo,capacidad_dias
0,14,cartago,destiladohp,6388.933125,0.0,510000.0,0.0,0.0,0.0,0.000000
1,85,buga,forraje,6237.256875,0.0,748000.0,0.0,0.0,0.0,0.000000
2,3,envigado,gluten,4309.328750,0.0,510000.0,0.0,0.0,0.0,0.000000
3,34,bogota,destiladohp,4199.458750,0.0,306000.0,0.0,0.0,0.0,0.000000
4,103,bmanga,gluten,3572.561250,0.0,425000.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
125,127,pimpollo,frijol,0.000000,5.0,238000.0,0.0,0.0,0.0,1000.000000
126,128,pimpollo,tgirasol,0.000000,0.0,238000.0,0.0,0.0,0.0,1000.000000
127,129,pimpollo,trigo,0.000000,0.0,0.0,0.0,0.0,0.0,1000.000000
128,9,envigado,trigo,298.975000,0.0,442000.0,0.0,530000.0,0.0,1772.723472
